In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/Users/juliankimura/Desktop/deepomics')
import deepomics.neuralnetwork as nn
from deepomics import learn, utils
from models import vae_model

import theano
import theano.tensor as T
from lasagne import layers

np.random.seed(247)   # for reproducibility

In [ ]:
filename = 'frey_rawface.mat'
data_path = '/Users/juliankimura/Desktop/data/FreyFaces'
matfile = loadmat(os.path.join(data_path, filename))
all_data = (matfile['ff'] / 255.).T

indices = np.arange(len(all_data))
np.random.shuffle(indices)
indices

width = 20
height = 28
X_train = all_data[indices[:1500]]
X_valid = all_data[indices[1500:]]

In [ ]:
plt.imshow(X_train[10].reshape([height,width]),cmap='gray')

In [ ]:
shape = (None, X_train.shape[1])
network, placeholders, optimization = vae_model.model(shape)

In [ ]:
# build neural network class
nnmodel = nn.NeuralNet(network, placeholders)
nnmodel.inspect_layers()

# set output file paths
output_name = 'Frey_faces'
results_path = utils.make_directory(data_path, 'Results')
results_path = utils.make_directory(results_path, output_name)
file_path = os.path.join(results_path, output_name)
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', file_path=file_path)

In [ ]:
# train model
learn.train_minibatch(nntrainer, data={'train': X_train, 'valid': X_valid}, 
                              batch_size=100, num_epochs=500, patience=10, verbose=0)

In [ ]:
num_steps = 10
samples = X_train[:num_steps*num_steps,:]
fig = plt.figure()
plt.imshow((samples.reshape(num_steps, num_steps, 28, 20)
                   .transpose(0, 2, 1, 3)
                   .reshape(num_steps*28, num_steps*20)), cmap='gray')
plt.axis('off')
fig.set_size_inches(10,10)

In [ ]:
generated_x = layers.get_output(nnmodel.network['decode_mu'], deterministic=True)
gen_fn = theano.function([nnmodel.placeholders['inputs']], generated_x)
num_grid = 10
k = 0
reconstructed_samples = []
for i in range(num_grid):
    for j in range(num_grid):
        reconstructed_samples.append(gen_fn(np.expand_dims(samples[k], axis=0)))
        k += 1
reconstructed_samples = np.array(reconstructed_samples)        
               
fig = plt.figure()
plt.imshow((reconstructed_samples.reshape(num_grid, num_grid, 28, 20)
                   .transpose(0, 2, 1, 3)
                   .reshape(num_grid*28, num_grid*20)), cmap='gray')
plt.axis('off')
fig.set_size_inches(10,10)

In [ ]:
z_var = T.vector()
generated_x = layers.get_output(nnmodel.network['decode_mu'], {nnmodel.network['encode_mu']: z_var}, deterministic=True)
gen_fn = theano.function([z_var], generated_x)
num_grid = 20
pos = np.linspace(-4, 4, num_grid)
generated_samples = []
for i in range(num_grid):
    for j in range(num_grid):
        z = np.asarray([pos[i], pos[j]], dtype=theano.config.floatX)
        generated_samples.append(gen_fn(z))
generated_samples = np.array(generated_samples)        
                
     
fig = plt.figure()
plt.imshow((generated_samples.reshape(num_grid, num_grid, 28, 20)
                   .transpose(0, 2, 1, 3)
                   .reshape(num_grid*28, num_grid*20)), cmap='gray')
plt.axis('off')
fig.set_size_inches(10,10)